# 🌊 Interactive Water Network Visualization

This notebook provides comprehensive interactive visualization capabilities for your water network graph database.

## Features:
- 🎯 **Interactive Plotly graphs** with hover details and zooming
- 🌐 **PyVis network visualization** with physics simulation
- 📊 **Network analysis** and statistics
- 🔍 **Component exploration** and path finding
- 📈 **Visual analytics** and reporting

## 1. Setup and Imports

In [1]:
# Required Modules

import sys
import os
import logging
import platform
from datetime import date, datetime

from dotenv import load_dotenv

import pandas as pd
import matplotlib.pyplot as plt
from neo4j import GraphDatabase

# Interactive visualization libraries
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from pyvis.network import Network
from IPython.display import HTML, display

# Jupyter display settings
import warnings
warnings.filterwarnings('ignore')

print(f"  System: {sys.platform}")
print(f"  Platform: {platform.platform()}")
print(f"  Python version: {platform.python_version()}")
print(f"  System Execution (Python) path: {'/'.join(sys.executable.strip('/').split('/')[-3:])}")
print(f"  Last update: {date.today().strftime('%Y-%m-%d')}")

  System: darwin
  Platform: macOS-15.6-arm64-arm-64bit
  Python version: 3.10.6
  System Execution (Python) path: .venv/bin/python
  Last update: 2025-08-14


In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## 2. Database Connection

**Update the connection parameters below with your Neo4j credentials:**

In [3]:
# Create connection
driver = GraphDatabase.driver(uri= os.environ.get("URI"),
                              auth=(os.environ.get("USER"),
                                    os.environ.get("PASSWORD")
                                    )
                                )

# Test connection
try:
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful!' AS message")
        print(f"🟢 {result.single()['message']}")
        
        # Check if we have data
        count_result = session.run("MATCH (n) RETURN count(n) as total")
        total_nodes = count_result.single()['total']
        print(f"📊 Found {total_nodes} nodes in the database")
        
        if total_nodes == 0:
            print("⚠️ No data found. Please run the setup notebook first to create sample data.")
            
except Exception as e:
    print(f"🔴 Connection failed: {e}")
    print("Please check your connection parameters and ensure Neo4j is running.")

🟢 Connection successful!
📊 Found 8 nodes in the database


## 3. Data Fetching Functions

In [3]:
def fetch_network_data(driver):
    """Fetch all nodes and relationships from Neo4j"""
    with driver.session() as session:
        # Get all nodes (using elementId instead of deprecated id)
        nodes_query = """
        MATCH (n) 
        RETURN elementId(n) as node_id, labels(n) as labels, properties(n) as props
        """
        nodes_result = session.run(nodes_query)
        nodes = []
        for record in nodes_result:
            node = {
                'id': record['node_id'],
                'label': record['labels'][0] if record['labels'] else 'Unknown',
                'properties': record['props']
            }
            nodes.append(node)
        
        # Get all relationships (using elementId instead of deprecated id)
        edges_query = """
        MATCH (n)-[r]->(m) 
        RETURN elementId(n) as source, elementId(m) as target, type(r) as relationship
        """
        edges_result = session.run(edges_query)
        edges = []
        for record in edges_result:
            edge = {
                'source': record['source'],
                'target': record['target'],
                'relationship': record['relationship']
            }
            edges.append(edge)
    
    return nodes, edges

# Test the function
nodes, edges = fetch_network_data(driver)
print(f"📊 Fetched {len(nodes)} nodes and {len(edges)} edges")
print(f"🏗️ Node types: {set(node['label'] for node in nodes)}")

📊 Fetched 8 nodes and 7 edges
🏗️ Node types: {'Tank', 'Valve', 'Pump', 'Pipe', 'Junction'}


## 4. Interactive Plotly Visualization

This creates a fully interactive graph with hover details, zooming, and component filtering.

In [4]:
def create_interactive_plotly_graph(driver):
    """Create an interactive graph using Plotly"""
    nodes, edges = fetch_network_data(driver)
    
    if not nodes:
        print("No data to visualize. Please create some nodes first.")
        return None
    
    # Create NetworkX graph
    G = nx.Graph()
    
    # Add nodes
    for node in nodes:
        G.add_node(node['id'], 
                  label=node['label'], 
                  component_id=node['properties'].get('id', 'Unknown'),
                  **node['properties'])
    
    # Add edges
    for edge in edges:
        G.add_edge(edge['source'], edge['target'], relationship=edge['relationship'])
    
    # Generate layout
    pos = nx.spring_layout(G, k=3, iterations=50)
    
    # Create edge traces
    edge_x = []
    edge_y = []
    
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=2, color='#888'),
        hoverinfo='none',
        mode='lines',
        name='Connections'
    )
    
    # Color scheme for different component types
    colors = {
        'Tank': '#1f77b4',      # Blue
        'Junction': '#ff7f0e',   # Orange  
        'Valve': '#2ca02c',      # Green
        'Pipe': '#d62728',       # Red
        'Pump': '#9467bd'        # Purple
    }
    
    # Create node traces by type
    node_traces = {}
    
    for node in nodes:
        node_type = node['label']
        if node_type not in node_traces:
            node_traces[node_type] = {
                'x': [], 'y': [], 'text': [], 'ids': [],
                'hovertext': []
            }
        
        x, y = pos[node['id']]
        node_traces[node_type]['x'].append(x)
        node_traces[node_type]['y'].append(y)
        node_traces[node_type]['ids'].append(node['id'])
        
        # Create hover text
        component_id = node['properties'].get('id', 'Unknown')
        hover_text = f"<b>{node_type}: {component_id}</b><br>"
        for key, value in node['properties'].items():
            if key != 'id':
                hover_text += f"{key}: {value}<br>"
        
        node_traces[node_type]['text'].append(component_id)
        node_traces[node_type]['hovertext'].append(hover_text)
    
    # Create Plotly traces
    traces = [edge_trace]
    
    for node_type, trace_data in node_traces.items():
        trace = go.Scatter(
            x=trace_data['x'], 
            y=trace_data['y'],
            mode='markers+text',
            text=trace_data['text'],
            textposition="middle center",
            hovertemplate="%{hovertext}<extra></extra>",
            hovertext=trace_data['hovertext'],
            marker=dict(
                size=20,
                color=colors.get(node_type, '#999999'),
                line=dict(width=2, color='white')
            ),
            name=node_type,
            showlegend=True
        )
        traces.append(trace)
    
    # Create layout with correct title format
    layout = go.Layout(
        title=dict(text="🌊 Interactive Water Network Graph", font=dict(size=16)),
        showlegend=True,
        hovermode='closest',
        margin=dict(b=20,l=5,r=5,t=40),
        annotations=[ dict(
            text="💡 Hover over nodes for details • Click legend to hide/show types • Drag to pan • Scroll to zoom",
            showarrow=False,
            xref="paper", yref="paper",
            x=0.005, y=-0.002,
            xanchor="left", yanchor="bottom",
            font=dict(color="grey", size=12)
        )],
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        plot_bgcolor='white'
    )
    
    fig = go.Figure(data=traces, layout=layout)
    return fig

In [5]:
# Create and display the interactive graph
print("🚀 Creating interactive Plotly graph...")
fig = create_interactive_plotly_graph(driver)
if fig:
    fig.show()
    print("✅ Interactive graph created! Try hovering, zooming, and clicking the legend.")

🚀 Creating interactive Plotly graph...


✅ Interactive graph created! Try hovering, zooming, and clicking the legend.


## 5. Network Analysis and Statistics

In [6]:
def analyze_network_properties(driver):
    """Analyze network properties and return statistics"""
    nodes, edges = fetch_network_data(driver)
    
    if not nodes:
        return None, None
    
    # Create NetworkX graph for analysis
    G = nx.Graph()
    for node in nodes:
        G.add_node(node['id'], **node['properties'])
    for edge in edges:
        G.add_edge(edge['source'], edge['target'])
    
    # Calculate network properties
    stats = {
        'total_nodes': len(nodes),
        'total_edges': len(edges),
        'density': nx.density(G),
        'is_connected': nx.is_connected(G),
        'number_of_components': nx.number_connected_components(G),
    }
    
    # Add clustering and diameter only if connected
    if nx.is_connected(G) and len(G) > 1:
        stats['average_clustering'] = nx.average_clustering(G)
        stats['diameter'] = nx.diameter(G)
    else:
        stats['average_clustering'] = 0
        stats['diameter'] = 'N/A (not connected)'
    
    # Node degree statistics
    degrees = dict(G.degree())
    if degrees:
        stats['average_degree'] = sum(degrees.values()) / len(degrees)
        stats['max_degree'] = max(degrees.values())
        stats['min_degree'] = min(degrees.values())
    else:
        stats['average_degree'] = 0
        stats['max_degree'] = 0
        stats['min_degree'] = 0
    
    # Component type distribution
    component_types = {}
    for node in nodes:
        node_type = node['label']
        component_types[node_type] = component_types.get(node_type, 0) + 1
    
    stats['component_distribution'] = component_types
    
    return stats, degrees

# Analyze the network
print("📈 Analyzing network properties...")
stats, degrees = analyze_network_properties(driver)

if stats:
    print("\n📋 Network Statistics:")
    print(f"  🔢 Total Nodes: {stats['total_nodes']}")
    print(f"  🔗 Total Edges: {stats['total_edges']}")
    print(f"  📊 Network Density: {stats['density']:.3f}")
    print(f"  🔄 Is Connected: {stats['is_connected']}")
    print(f"  📐 Average Degree: {stats['average_degree']:.2f}")
    print(f"  🎯 Clustering: {stats['average_clustering']:.3f}")
    print(f"  📏 Diameter: {stats['diameter']}")
    
    print(f"\n🏗️ Component Distribution:")
    for comp_type, count in stats['component_distribution'].items():
        print(f"  📦 {comp_type}: {count}")
else:
    print("No data to analyze.")

📈 Analyzing network properties...

📋 Network Statistics:
  🔢 Total Nodes: 8
  🔗 Total Edges: 7
  📊 Network Density: 0.250
  🔄 Is Connected: True
  📐 Average Degree: 1.75
  🎯 Clustering: 0.000
  📏 Diameter: 6

🏗️ Component Distribution:
  📦 Tank: 1
  📦 Junction: 2
  📦 Valve: 2
  📦 Pump: 1
  📦 Pipe: 2


## 9. Custom Query Interface

Run your own custom queries here.

In [10]:
# Custom query area - modify the queries below to explore your network

def run_custom_query(query, description="Custom Query"):
    """Run a custom Cypher query"""
    print(f"\n🔍 {description}:")
    print(f"📝 Query: {query}")
    print("-" * 50)
    
    try:
        with driver.session() as session:
            result = session.run(query)
            records = list(result)
            
            if records:
                for i, record in enumerate(records, 1):
                    print(f"  {i}. {dict(record)}")
                print(f"\n✅ Found {len(records)} results")
            else:
                print("  ❌ No results found")
                
    except Exception as e:
        print(f"  🔴 Error: {e}")

# Example custom queries - modify these as needed
print("🎯 Running Custom Query Examples:")

# Query 1: Find components by type
run_custom_query(
    "MATCH (v:Valve) RETURN v.id, v.status, v.type",
    "Find all valves with their status"
)

# Query 2: Find old components
run_custom_query(
    "MATCH (n) WHERE exists(n.installDate) AND n.installDate < date('2015-01-01') RETURN n.id, labels(n), n.installDate",
    "Find components installed before 2015"
)

# Query 3: Find highly connected components
run_custom_query(
    "MATCH (n)-[r]-() RETURN n.id, labels(n), count(r) as connections ORDER BY connections DESC LIMIT 3",
    "Find most connected components"
)

print("\n💡 Add your own queries in the cell below!")

🎯 Running Custom Query Examples:

🔍 Find all valves with their status:
📝 Query: MATCH (v:Valve) RETURN v.id, v.status, v.type
--------------------------------------------------
  1. {'v.id': 'VLV001', 'v.status': 'Open', 'v.type': 'Gate Valve'}
  2. {'v.id': 'VLV002', 'v.status': 'Closed', 'v.type': 'Ball Valve'}

✅ Found 2 results

🔍 Find components installed before 2015:
📝 Query: MATCH (n) WHERE exists(n.installDate) AND n.installDate < date('2015-01-01') RETURN n.id, labels(n), n.installDate
--------------------------------------------------
  🔴 Error: {code: Neo.ClientError.Statement.SyntaxError} {message: The property existence syntax `... exists(variable.property)` is no longer supported. Please use `variable.property IS NOT NULL` instead. (line 1, column 17 (offset: 16))
"MATCH (n) WHERE exists(n.installDate) AND n.installDate < date('2015-01-01') RETURN n.id, labels(n), n.installDate"
                 ^}

🔍 Find most connected components:
📝 Query: MATCH (n)-[r]-() RETURN n.id, 

In [11]:
# 🎮 YOUR CUSTOM QUERIES HERE
# Copy and modify the run_custom_query calls above to explore your network

# Example: Uncomment and modify these queries

# Find all pipes and their materials
run_custom_query(
    "MATCH (p:Pipe) RETURN p.id, p.material, p.diameter, p.length ORDER BY p.length DESC",
    "All pipes sorted by length"
)

# Find components in a specific area
# run_custom_query(
#     "MATCH (n) WHERE exists(n.location) AND n.location CONTAINS 'North' RETURN n.id, labels(n), n.location",
#     "Components in North area"
# )

# Find flow paths from tanks
# run_custom_query(
#     "MATCH path = (t:Tank)-[:FEEDS*1..3]->(n) RETURN t.id as tank, collect(n.id) as fed_components",
#     "Components fed by tanks (up to 3 hops)"
# )

print("✏️ Add your queries above by uncommenting and modifying the examples!")


🔍 All pipes sorted by length:
📝 Query: MATCH (p:Pipe) RETURN p.id, p.material, p.diameter, p.length ORDER BY p.length DESC
--------------------------------------------------
  1. {'p.id': 'PIP001', 'p.material': 'Cast Iron', 'p.diameter': 12, 'p.length': 500.0}
  2. {'p.id': 'PIP002', 'p.material': 'PVC', 'p.diameter': 8, 'p.length': 300.0}

✅ Found 2 results
✏️ Add your queries above by uncommenting and modifying the examples!


## 10. Summary and Next Steps

In [12]:
# Final summary
print("🎉 INTERACTIVE WATER NETWORK VISUALIZATION COMPLETE!")
print("=" * 60)

print("\n📊 What you've created:")
print("  ✅ Interactive Plotly graph with hover details")
print("  ✅ Network analysis and statistics dashboard")
print("  ✅ PyVis interactive network (HTML file)")
print("  ✅ Component exploration functions")
print("  ✅ Custom query interface")

print("\n🎯 Key interactive features:")
print("  🖱️ Hover over nodes for component details")
print("  🔍 Zoom and pan in the graphs")
print("  👁️ Click legend to hide/show component types")
print("  🌐 Open HTML file for physics-based interaction")
print("  🔍 Query specific components and paths")

if 'html_file' in locals():
    print(f"\n📁 Files created:")
    print(f"  🌐 {html_file} - Interactive network (open in browser)")

print("\n🚀 Next steps:")
print("  1. Explore the interactive visualizations above")
print("  2. Try modifying the custom queries")
print("  3. Add more components to your network")
print("  4. Experiment with different network layouts")
print("  5. Share the HTML file for interactive presentations")

print("\n💡 Pro tips:")
print("  • Use this notebook for water network presentations")
print("  • Modify colors and layouts to match your company branding")
print("  • Add real-world coordinates for geographic visualization")
print("  • Integrate with real-time data for live monitoring")

print("\n🌊 Happy water network exploration!")

🎉 INTERACTIVE WATER NETWORK VISUALIZATION COMPLETE!

📊 What you've created:
  ✅ Interactive Plotly graph with hover details
  ✅ Network analysis and statistics dashboard
  ✅ PyVis interactive network (HTML file)
  ✅ Component exploration functions
  ✅ Custom query interface

🎯 Key interactive features:
  🖱️ Hover over nodes for component details
  🔍 Zoom and pan in the graphs
  👁️ Click legend to hide/show component types
  🌐 Open HTML file for physics-based interaction
  🔍 Query specific components and paths

📁 Files created:
  🌐 water_network_interactive.html - Interactive network (open in browser)

🚀 Next steps:
  1. Explore the interactive visualizations above
  2. Try modifying the custom queries
  3. Add more components to your network
  4. Experiment with different network layouts
  5. Share the HTML file for interactive presentations

💡 Pro tips:
  • Use this notebook for water network presentations
  • Modify colors and layouts to match your company branding
  • Add real-world

## 📝 Notes and Customization

Use this section to add your own notes, customizations, or additional analysis:

### Custom Color Schemes
You can modify the color dictionary in the visualization functions:
```python
colors = {
    'Tank': '#your_color_here',
    'Junction': '#your_color_here', 
    # ... etc
}
```

### Adding Real Coordinates
If you have GPS coordinates, you can create geographic visualizations using Plotly's mapping features.

### Performance Notes
- For large networks (>1000 nodes), consider using network sampling
- PyVis may be slow with very large networks
- Use filtering to focus on specific network areas

### Company Presentation Tips
- Save key visualizations as images for slides
- Use the HTML file for interactive demos
- Customize titles and annotations with company branding
- Focus on business value: cost savings, efficiency, risk reduction